<div style="background-color:#000;"><img src="pqn.png"></img></div>

This code retrieves financial data and calculates the cost and implications of an options straddle strategy for a given stock. It fetches the earnings calendar, stock price, and options chain for a specific symbol. It then identifies the at-the-money (ATM) call and put options expiring shortly after the earnings report. The cost of the straddle and the implied daily price movement are calculated and printed. This is useful for traders and investors to understand potential price movements around earnings announcements.

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
from openbb import obb

In [ ]:
obb.user.preferences.output_type = "dataframe"
obb.user.credentials.nasdaq_api_key = "PLACE_HOLDER"

Retrieve earnings calendar data from OpenBB for the next 14 days from today

In [ ]:
earnings_calendar = obb.equity.calendar.earnings(
    start_date=(datetime.now() + timedelta(days=1)).date(),
    end_date=(datetime.now() + timedelta(days=14)).date(),
    provider="nasdaq"
)

Define the symbol for Costco (COST) and fetch the latest stock price

In [ ]:
symbol = "COST"
last_price = (
    obb
    .equity
    .price
    .quote(symbol, provider="yfinance")
    .T
    .loc["last_price", 0]
)

Retrieve options chains for the specified symbol from OpenBB

In [ ]:
options = obb.derivatives.options.chains(symbol, provider="cboe")

Filter options chain for contracts expiring on the specified date (shortly after earnings)

In [ ]:
expiration = datetime(2024, 3, 8).date()
chain = options.query("`expiration` == @expiration")

Extract strike prices and identify the ATM strike for calls and puts

In [ ]:
strikes = chain.strike.to_frame()

In [ ]:
call_strike = (
    strikes
    .loc[strikes.query("`strike` > @last_price").idxmin()]["strike"]
    .iloc[0]
)

In [ ]:
atm_call = chain.query("`strike` == @call_strike and `option_type` == 'call'")
atm_put = chain.query("`strike` == @call_strike and `option_type` == 'put'")

Concatenate ATM call and put options and calculate the total ask price for the straddle

In [ ]:
atm = pd.concat([atm_call, atm_put])
straddle_price = round(atm.ask.sum(), 2)

Calculate the implied daily price movement based on the straddle price and time to expiration

In [ ]:
days = (atm.expiration.iloc[0] - datetime.now().date()).days + 1
implied_move = ((1 + straddle_price / last_price) ** (1 / days) - 1) * 100

Calculate the upper and lower breakeven prices for the straddle

In [ ]:
upper_price = round((last_price * (1 + (straddle_price / last_price))), 2)
lower_price = round((last_price * (1 - (straddle_price / last_price))), 2)

Print the calculated straddle cost, its percentage of the share price, breakeven prices, and implied daily move

In [ ]:
print(
    f"Cost of Straddle: ${straddle_price}"
    f"\nCost as a % of Share Price: {round((straddle_price / last_price) * 100, 4)}%"
    f"\nUpper Breakeven Price: ${upper_price}"
    f"\nLower Breakeven Price: ${lower_price}\n"
    f"\nImplied Daily Move: {round(implied_move, 4)}%\n"
)

Output the number of days until the specified options expiration date

In [ ]:
days

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.